In [1]:
!pip install dvc dvc[gdrive]

     |████████████████████████████████| 666 kB 6.6 MB/s 
     |████████████████████████████████| 296 kB 60.7 MB/s 
     |████████████████████████████████| 211 kB 34.3 MB/s 
     |████████████████████████████████| 170 kB 59.9 MB/s 
     |████████████████████████████████| 44 kB 3.2 MB/s 
     |████████████████████████████████| 4.6 MB 41.2 MB/s 
     |████████████████████████████████| 530 kB 47.3 MB/s 
     |████████████████████████████████| 109 kB 57.5 MB/s 
     |████████████████████████████████| 40 kB 17 kB/s 
     |████████████████████████████████| 119 kB 68.5 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 201 kB 60.0 MB/s 
     |████████████████████████████████| 2.6 MB 42.6 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
     |██████████████████████

In [19]:
# Data Version Control
import os
import dvc.api
from io import StringIO,BytesIO
# Other
import warnings
warnings.filterwarnings('ignore')
# Your dependencies ... 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

%matplotlib inline

In [2]:
# Data Version Control
import os
import dvc.api
from io import StringIO,BytesIO

# Your dependencies ... 
import pandas as pd

In [3]:
def get_dvc_file(file_path: str, github_tag='v1.1_Gdrive_remote', 
                 pat_path='.github_pat', repo='Explore-AI/EXPLORE.Utilities.Epl.Core.git'):
  """Utility function to convert a file path into DVC-based
     file stream for the underlying data. 

    Args:
        file_path (str): Path to the data file. Should be relative to the 
                         base of the Git repository. 
        github_tag (str): The GitHub tag associated with the version 
                          of the data being used.
        pat_path (str): Path to a file containing your GitHub personal
                        access token (PAT).
        repo (str): The GitHub repo associated with DVC where the data is 
                    being tracked. 
  """
  # Retrieve GitHub secrets file
  try:
    with open(pat_path, "r") as f: 
      pat_secret = f.readline()
  except: 
    raise FileNotFoundError(f"The given path to your GitHub PAT file: '{pat_path}', doesn't exist")
  
  # Choose reading method based-on input file type
  file_extension = file_path.split('.')[-1]

  if file_extension in ['csv']:
    read_mode = 'r'
    reader = lambda x: StringIO(x)
  else:
    read_mode = 'rb'
    reader = lambda x: BytesIO(x)

  file_stream = dvc.api.read(
      path=file_path,
      repo=f'https://{pat_secret}@github.com/{repo}',
      rev=github_tag,
      mode=read_mode
  )
  file_stream = reader(file_stream)
  return file_stream

In [9]:
#Filtered raw data
raw_filtered = pd.read_csv(get_dvc_file('enrich/water/synthetic_data/validation/data/filtered_raw.csv','v1.3_raw_filter_data'))

In [10]:
raw_filtered.isnull().sum()

timestamp         0
tag_name          0
value        130167
dtype: int64

In [11]:
def fill_empty(df, column):
    """
    This function replaces empty and null values using the forward fill method
    """
    changed = df[column].isnull().sum()
    df[column].fillna((df[column].mean()), inplace=True)
    
    return 'Number of entries filled: '+str(changed)

In [12]:
fill_empty(raw_filtered, 'value')

'Number of entries filled: 130167'

In [13]:
raw_filtered.isnull().sum()

timestamp    0
tag_name     0
value        0
dtype: int64

In [14]:
raw_filtered.head()

,timestamp,tag_name,value
0,2021-05-31T23:45:00.000Z,DLM:14336-FLOW,30.410847
1,2021-06-01T00:00:00.000Z,DLM:14336-FLOW,29.162193
2,2021-06-01T00:15:00.000Z,DLM:14336-FLOW,28.678840
3,2021-06-01T00:30:00.000Z,DLM:14336-FLOW,27.416758
4,2021-06-01T00:45:00.000Z,DLM:14336-FLOW,26.906553


In [15]:
raw_filtered.set_index('timestamp', inplace=True)

In [16]:
raw_filtered.head()

,tag_name,value
timestamp,,
2021-05-31T23:45:00.000Z,DLM:14336-FLOW,30.410847
2021-06-01T00:00:00.000Z,DLM:14336-FLOW,29.162193
2021-06-01T00:15:00.000Z,DLM:14336-FLOW,28.678840
2021-06-01T00:30:00.000Z,DLM:14336-FLOW,27.416758
2021-06-01T00:45:00.000Z,DLM:14336-FLOW,26.906553


In [17]:
raw_filtered.describe()

,value
count,2.525889e+06
mean,1.220961e+01
std,1.970281e+01
min,-1.425432e+02
25%,3.000000e-01
50%,5.729991e+00
75%,1.735883e+01
max,2.821402e+02
